In [ ]:
# -*- coding: utf-8 -*-
# ipynb 공통 유틸: 경로, split 생성, config 안전 패치

import json
import random
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import pandas as pd

from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules


# -----------------------
# 1) 고정 경로 (네 환경 기준)
# -----------------------

# mmdetection 루트 (configs 탐색용)
MMD_ROOT = Path("/data/ephemeral/home/model/baseline/mmdetection")

# 데이터 루트
FULL_DATA_ROOT = Path("/data/ephemeral/home/model/dataset")
TRAIN_JSON_FULL = FULL_DATA_ROOT / "train.json"
TEST_JSON_FULL  = FULL_DATA_ROOT / "test.json"

# 샘플 제출 폴더
SAMPLE_SUB_DIR = Path("/data/ephemeral/home/model/sample_submission")

# 단일 모델 실험 산출물 저장 위치
WORK_DIR_ROOT = Path("/data/ephemeral/home/model/work_dirs_single")
WORK_DIR_ROOT.mkdir(parents=True, exist_ok=True)

# 이미지 스케일 고정
IMAGE_SCALE = (1024, 1024)

# 클래스 10개 (대회 공지 기준 순서)
CLASSES = (
    "General trash",
    "Paper",
    "Paper pack",
    "Metal",
    "Glass",
    "Plastic",
    "Styrofoam",
    "Plastic bag",
    "Battery",
    "Clothing",
)

# split 설정
RANDOM_SEED = 42
TRAIN_RATIO = 0.9   # 0.8/0.2보다 학습 데이터를 늘려 단일 모델 성능 안정성을 노림


print("MMD_ROOT:", MMD_ROOT)
print("FULL_DATA_ROOT:", FULL_DATA_ROOT)
print("TRAIN_JSON_FULL:", TRAIN_JSON_FULL)
print("TEST_JSON_FULL :", TEST_JSON_FULL)
print("SAMPLE_SUB_DIR :", SAMPLE_SUB_DIR)
print("WORK_DIR_ROOT  :", WORK_DIR_ROOT)


# -----------------------
# 2) COCO json 로드/저장
# -----------------------

def load_coco(json_path: Path) -> Dict:
    with open(json_path, "r") as f:
        return json.load(f)

def save_coco(data: Dict, json_path: Path) -> None:
    json_path.parent.mkdir(parents=True, exist_ok=True)
    with open(json_path, "w") as f:
        json.dump(data, f)


# -----------------------
# 3) train/val split 생성
# -----------------------

def make_train_val_split(
    src_json: Path,
    out_dir: Path,
    train_ratio: float = 0.9,
    seed: int = 42
) -> Dict[str, Path]:
    """
    이미지 id 기준 랜덤 분할.
    detection에서 가장 기본적이고 안전한 방식.
    """
    random.seed(seed)

    data = load_coco(src_json)
    images = data["images"]
    anns = data["annotations"]

    img_ids = [img["id"] for img in images]
    random.shuffle(img_ids)

    split_idx = int(len(img_ids) * train_ratio)
    train_ids = set(img_ids[:split_idx])
    val_ids   = set(img_ids[split_idx:])

    def _filter(ids: set):
        # ids에 해당하는 image만 남김
        imgs = [img for img in images if img["id"] in ids]
        img_set = {img["id"] for img in imgs}
        # 해당 이미지에 매칭되는 annotation만 남김
        filtered_anns = [ann for ann in anns if ann["image_id"] in img_set]
        return {**data, "images": imgs, "annotations": filtered_anns}

    out_dir.mkdir(parents=True, exist_ok=True)
    train_json = out_dir / "train_split.json"
    val_json   = out_dir / "val_split.json"

    save_coco(_filter(train_ids), train_json)
    save_coco(_filter(val_ids), val_json)

    return {"train": train_json, "val": val_json}


# -----------------------
# 4) pipeline 스케일 고정
# -----------------------

def set_img_scale(pipeline, scale):
    """
    Resize / RandomResize / RandomChoiceResize 등을 1024로 통일.
    """
    for t in pipeline:
        if isinstance(t, list):
            set_img_scale(t, scale)
            continue
        if not isinstance(t, dict):
            continue

        if t.get("type") in ("Resize", "RandomResize", "RandomChoiceResize"):
            if "scale" in t:
                t["scale"] = scale
            if "img_scale" in t:
                t["img_scale"] = scale
            if "scales" in t:
                t["scales"] = [scale]

        if "transforms" in t:
            set_img_scale(t["transforms"], scale)


# -----------------------
# 5) num_classes 재귀 고정
# -----------------------

def set_num_classes(model_cfg, num_classes: int):
    """
    다양한 head 구조에서 num_classes를 재귀적으로 10으로 맞춤.
    """
    if isinstance(model_cfg, dict):
        if "num_classes" in model_cfg:
            model_cfg["num_classes"] = num_classes
        for v in model_cfg.values():
            set_num_classes(v, num_classes)
    elif isinstance(model_cfg, list):
        for v in model_cfg:
            set_num_classes(v, num_classes)


# -----------------------
# 6) 샘플 csv 자동 선택
# -----------------------

def pick_sample_csv(sample_dir: Path) -> Path:
    """
    - 이름에 mmdetection 포함된 샘플을 우선
    - 없으면 첫 번째 csv
    """
    csvs = sorted(sample_dir.glob("*.csv"))
    if not csvs:
        raise FileNotFoundError(f"샘플 제출 csv가 없습니다: {sample_dir}")

    for c in csvs:
        if "mmdetection" in c.name.lower():
            return c

    return csvs[0]


# -----------------------
# 7) base config 자동 탐색
# -----------------------

def pick_best_single_base_cfg() -> Path:
    """
    앙상블에 없던 단일 후보로 RTMDet 계열을 우선 시도.
    환경에 따라 config 파일명이 다를 수 있어
    여러 후보 중 '존재하는 첫 번째'를 선택.
    """
    candidates = [
        "configs/rtmdet/rtmdet_l_8xb32-300e_coco.py",
        "configs/rtmdet/rtmdet_m_8xb32-300e_coco.py",
        "configs/rtmdet/rtmdet_s_8xb32-300e_coco.py",
        # 혹시 RTMDet가 없다면 2순위 후보
        "configs/yolox/yolox_l_8xb8-300e_coco.py",
        "configs/yolox/yolox_m_8xb8-300e_coco.py",
    ]

    for rel in candidates:
        p = MMD_ROOT / rel
        if p.exists():
            print("Selected base cfg:", p)
            return p

    # 마지막 안전장치: 그래도 없으면 에러로 명확히 알려줌
    raise FileNotFoundError(
        "RTMDet/YOLOX 후보 config를 찾지 못했습니다. "
        "MMD_ROOT/configs 아래 실제 파일명을 확인해 주세요."
    )


MMD_ROOT: /data/ephemeral/home/model/baseline/mmdetection
FULL_DATA_ROOT: /data/ephemeral/home/model/dataset
TRAIN_JSON_FULL: /data/ephemeral/home/model/dataset/train.json
TEST_JSON_FULL : /data/ephemeral/home/model/dataset/test.json
SAMPLE_SUB_DIR : /data/ephemeral/home/model/sample_submission
WORK_DIR_ROOT  : /data/ephemeral/home/model/work_dirs_single
